In [1]:
!pip install git+https://github.com/openai/CLIP.git onnxruntime onnx-simplifier

  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\User\AppData\Local\Temp\pip-req-build-w_ti9mj6'

[notice] A new release of pip is available: 24.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Cloning https://github.com/openai/CLIP.git to c:\users\user\appdata\local\temp\pip-req-build-w_ti9mj6
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


In [2]:
import clip
import time
import torch
import onnx
import onnxruntime as ort
from onnxsim import simplify

from typing import Tuple

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
CLIP_BACKBONE = 'RN50'
CLIP_ONNX_EXPORT_PATH = 'clip_resnet.onnx'
CLIP_ONNX_EXPORT_PATH_SIMP = 'clip_resnet_simplified.onnx'

ONNX_INPUT_NAMES = ["IMAGE", "TEXT"]
ONNX_OUTPUT_NAMES = ["LOGITS_PER_IMAGE", "LOGITS_PER_TEXT"]
ONNX_DYNAMIC_AXES = {
    "IMAGE": {
        0: "image_batch_size",
    },
    "TEXT": {
        0: "text_batch_size",
    },
    "LOGITS_PER_IMAGE": {
        0: "image_batch_size",
        1: "text_batch_size",
    },
    "LOGITS_PER_TEXT": {
        0: "text_batch_size",
        1: "image_batch_size",
    },
}

In [5]:
def measure_mean_time_no_warmup(
    func, 
    func_inputs, 
    num_iters=250
) -> float:
    start_time = time.perf_counter()
    for _ in range(num_iters):
        func(*func_inputs)
    return (time.perf_counter() - start_time) / num_iters

def load_clip(backbone='RN50', device='cpu') -> Tuple[clip.model.CLIP, Tuple[torch.Tensor, torch.Tensor]]:
    pytorch_model, pre = clip.load(backbone)
    
    # Переместим модель на нужное устройство
    pytorch_model = pytorch_model.to(device)
    
    # Генерируем dummy inputs и перемещаем их на то же устройство
    npx = pytorch_model.visual.input_resolution
    dummy_image = torch.randn(10, 3, npx, npx).to(device)
    dummy_texts = clip.tokenize(["quick brown fox", "lorem ipsum"]).to(device)
    
    return pytorch_model, (dummy_image, dummy_texts)

def export_onnx(
    model, 
    inputs, 
    input_names,
    output_names,
    dynamic_axes,
    export_path
) -> None:
    torch.onnx.export(
        model=model, 
        args=inputs, 
        f=export_path, 
        export_params=True,
        input_names=input_names,
        output_names=output_names,
        opset_version=14,
        dynamic_axes=dynamic_axes
    )

In [6]:
pytorch_model, dummy_input = load_clip(backbone=CLIP_BACKBONE, device=device)
pytorch_model.eval()
pytorch_model.float()

export_onnx(
    model=pytorch_model,
    inputs=dummy_input,
    input_names=ONNX_INPUT_NAMES,
    output_names=ONNX_OUTPUT_NAMES,
    dynamic_axes=ONNX_DYNAMIC_AXES,
    export_path=CLIP_ONNX_EXPORT_PATH,
)

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\onnx\symbolic_opset9.py:5858: UserWarning: Exporting aten::index operator of advanced indexing in opset 14 is achieved by combination of multiple ONNX operators, including Reshape, Transpose, Concat, and Gather. If indices include negative values, the exported graph will produce incorrect results.
  warnings.warn(


In [7]:
# run checks
onnx_model = onnx.load(CLIP_ONNX_EXPORT_PATH)
onnx.checker.check_model(onnx_model)

# run additional checks and simplify
model_simp, check = simplify(onnx_model, skip_fuse_bn=True)
assert check, "Simplified ONNX model could not be validated"
onnx.save(model_simp, CLIP_ONNX_EXPORT_PATH_SIMP)

In [8]:
ort_sess = ort.InferenceSession(CLIP_ONNX_EXPORT_PATH_SIMP)

In [12]:
with torch.no_grad():
    pytorch_output = pytorch_model(*dummy_input)

onnx_output = ort_sess.run(
    ONNX_OUTPUT_NAMES, 
    {
        "IMAGE": dummy_input[0].cpu().numpy(), 
        "TEXT": dummy_input[1].cpu().numpy()
    }
)

# Сравним и выведем значения
for i, (pt_pred, onnx_pred) in enumerate(zip(pytorch_output, onnx_output)):
    pt_pred_cpu = pt_pred.cpu()  # Переводим выходной тензор PyTorch на CPU для сравнения
    onnx_pred_tensor = torch.tensor(onnx_pred)  # Преобразуем выходные данные ONNX в тензор
    
    # Вывод для отладки
    print(f"PyTorch Output {i}:\n", pt_pred_cpu)
    print(f"ONNX Output {i}:\n", onnx_pred_tensor)
    print(f"Difference (absolute):\n", torch.abs(pt_pred_cpu - onnx_pred_tensor).max())

    # Проверка с увеличенной погрешностью
    assert torch.allclose(pt_pred_cpu, onnx_pred_tensor, atol=1e-4, rtol=1e-3), f"Outputs differ at index {i}"


PyTorch Output 0:
 tensor([[14.0831, 18.9359],
        [13.6810, 18.7863],
        [14.5861, 18.8607],
        [14.2820, 19.0347],
        [14.0733, 18.8717],
        [14.4109, 19.1839],
        [14.3502, 19.3408],
        [14.3873, 18.8750],
        [14.2595, 19.1647],
        [14.2893, 19.0929]])
ONNX Output 0:
 tensor([[14.0844, 18.9358],
        [13.6807, 18.7858],
        [14.5882, 18.8621],
        [14.2823, 19.0324],
        [14.0748, 18.8727],
        [14.4093, 19.1830],
        [14.3506, 19.3406],
        [14.3906, 18.8766],
        [14.2585, 19.1637],
        [14.2909, 19.0933]])
Difference (absolute):
 tensor(0.0033)
PyTorch Output 1:
 tensor([[14.0831, 13.6810, 14.5861, 14.2820, 14.0733, 14.4109, 14.3502, 14.3873,
         14.2595, 14.2893],
        [18.9359, 18.7863, 18.8607, 19.0347, 18.8717, 19.1839, 19.3408, 18.8750,
         19.1647, 19.0929]])
ONNX Output 1:
 tensor([[14.0844, 13.6807, 14.5882, 14.2823, 14.0748, 14.4093, 14.3506, 14.3906,
         14.2585, 14.2909],
 

In [14]:
with torch.no_grad():
    pytorch_mean_time = measure_mean_time_no_warmup(func=pytorch_model, func_inputs=dummy_input)
onnx_runtime_mean_time = measure_mean_time_no_warmup(func=ort_sess.run, func_inputs=(["LOGITS_PER_IMAGE", "LOGITS_PER_TEXT"], {"IMAGE": dummy_input[0].cpu().numpy(), "TEXT": dummy_input[1].cpu().numpy()}))

print(f'PyTorch mean time: {round(pytorch_mean_time, 3)} sec\nONNX Runtime mean time: {round(onnx_runtime_mean_time, 3)} sec\nBoost from PT -> ONNX (%) {100*round(1 - onnx_runtime_mean_time/pytorch_mean_time, 2)}')

PyTorch mean time: 0.033 sec
ONNX Runtime mean time: 0.293 sec
Boost from PT -> ONNX (%) -783.0


# что мне с этим добром делать

## Подготовка изображений и текстов
Для модели CLIP входными данными являются изображения и текст. Вам нужно убедиться, что они в правильном формате.

Изображения должны быть преобразованы в тензоры PyTorch размера (batch_size, 3, H, W), где H и W — это разрешение, которое ожидает CLIP (обычно 224x224 или 336x336 для предобученных моделей).
Тексты должны быть токенизированы в формат, поддерживаемый CLIP.

## Преобразование данных

In [ ]:
import clip
from PIL import Image

# Подготовка собственных данных
image_path = 'path_to_your_image.jpg'
text_inputs = ["это мой текст", "еще один пример текста"]

# Загружаем и подготавливаем изображение
image = Image.open(image_path).convert("RGB")
image_input = preprocess(image).unsqueeze(0).to(device)  # Добавляем измерение для batch и отправляем на device

# Токенизация текстов для CLIP
text_input = clip.tokenize(text_inputs).to(device)


## инференс

In [ ]:
# PyTorch инференс
with torch.no_grad():
    pytorch_output = pytorch_model(image_input, text_input)

# ONNX инференс
onnx_output = ort_sess.run(
    ONNX_OUTPUT_NAMES, 
    {
        "IMAGE": image_input.cpu().numpy(),
        "TEXT": text_input.cpu().numpy()
    }
)

## работа с результатами

In [ ]:
# Преобразуем результаты в тензоры для дальнейшего использования
pytorch_logits_image, pytorch_logits_text = pytorch_output
onnx_logits_image, onnx_logits_text = [torch.tensor(out) for out in onnx_output]

# Расчёт сходства (например, скалярное произведение или другой метод) между изображениями и текстами
similarity_scores_pytorch = pytorch_logits_image @ pytorch_logits_text.T
similarity_scores_onnx = onnx_logits_image @ onnx_logits_text.T

# Выводим или используем результаты
print("Сходство для PyTorch модели:\n", similarity_scores_pytorch)
print("Сходство для ONNX модели:\n", similarity_scores_onnx)


In [1]:
import torch
import clip
from PIL import Image


# Загрузка модели и предобученных весов
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Подготовка изображения и текста
image = preprocess(Image.open("image.jpg")).unsqueeze(0).to(device)
texts = clip.tokenize(["a Guitar image", "A photo of a dog"]).to(device)

# Инференс
with torch.no_grad():
    # Получение эмбеддингов изображения и текста
    image_features = model.encode_image(image)
    text_features = model.encode_text(texts)

    # Нормализация эмбеддингов для более корректного вычисления сходства
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # Вычисление косинусного сходства
    similarities = (100.0 * image_features @ text_features.T).softmax(dim=-1)

# Вывод результатов
probs = similarities.squeeze().cpu().numpy()
for i, prob in enumerate(probs):
    print(f"Вероятность, что изображение соответствует тексту {i + 1}: {prob:.2%}")


Вероятность, что изображение соответствует тексту 1: 100.00%
Вероятность, что изображение соответствует тексту 2: 0.01%


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
